In [2]:
import os
import cv2
import numpy as np

page_img_path = "image.png"
train_chart_folder = "train/cropped"
output_path = "best_matched_crop.png"

# Load image paths from folder
def load_image_paths(folder):
    image_paths = []
    for filename in os.listdir(folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            image_paths.append(os.path.join(folder, filename))
    return image_paths

# Function to split the image into two halves (vertically)
def split_image(input_img):
    ih, iw = input_img.shape[:2]
    mid = iw // 2  # Divide the width in half
    left_half = input_img[:, :mid]  # Left half of the image
    right_half = input_img[:, mid:]  # Right half of the image
    return left_half, right_half

# Function to find the matched region
def find_matched_region(input_img, chart_img):
    ih, iw = input_img.shape[:2]
    ch, cw = chart_img.shape[:2]

    if ch > ih or cw > iw:
        print(f"⚠️ Skipping oversized template: Chart size {chart_img.shape} > Input size {input_img.shape}")
        return None, -1

    input_gray = cv2.cvtColor(input_img, cv2.COLOR_RGB2GRAY)
    chart_gray = cv2.cvtColor(chart_img, cv2.COLOR_RGB2GRAY)

    result = cv2.matchTemplate(input_gray, chart_gray, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    h, w = chart_img.shape[:2]
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    matched_region = input_img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]

    return matched_region, max_val

# Load input image and convert to RGB
input_img = cv2.imread(page_img_path)
if input_img is None:
    raise FileNotFoundError(f"Page image not found: {page_img_path}")
input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)

chart_image_paths = load_image_paths(train_chart_folder)

best_score = -1
best_crop = None
best_match_path = None

# Split the input image if it's too large
left_half, right_half = split_image(input_img)

# Process both halves (left and right)
for part, part_name in zip([left_half, right_half], ["Left", "Right"]):
    print(f"Processing {part_name} half...")

    for path in chart_image_paths:
        chart_img = cv2.imread(path)
        if chart_img is None:
            print(f"⚠️ Could not load image: {path}")
            continue

        chart_img = cv2.cvtColor(chart_img, cv2.COLOR_BGR2RGB)

        cropped_region, score = find_matched_region(part, chart_img)
        if score == -1:
            continue  

        print(f"✅ Matched {os.path.basename(path)} with score: {score:.4f} in {part_name} half")

        if score > best_score:
            best_score = score
            best_crop = cropped_region
            best_match_path = path

# If a match is found, save and display the result
if best_crop is not None:
    print(f"\n🎯 Best match: {os.path.basename(best_match_path)} with score: {best_score:.4f}")

    cropped_bgr = cv2.cvtColor(best_crop, cv2.COLOR_RGB2BGR)

    cv2.namedWindow("Best Matched Region", cv2.WINDOW_NORMAL)
    cv2.imshow("Best Matched Region", cropped_bgr)

    h, w = cropped_bgr.shape[:2]
    cv2.resizeWindow("Best Matched Region", w, h)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the cropped image
    cv2.imwrite(output_path, cropped_bgr)
    print(f"💾 Cropped region saved to {output_path}")
else:
    print("❌ No suitable match found.")


Processing Left half...
⚠️ Skipping oversized template: Chart size (390, 2800, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (390, 2800, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (470, 3400, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (470, 3400, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (470, 3400, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (470, 3400, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (470, 3400, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (470, 3400, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (470, 3400, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (470, 3400, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart size (470, 3400, 3) > Input size (903, 401, 3)
⚠️ Skipping oversized template: Chart siz

In [ ]:
# Example for processing the cropped region
cropped_tensor = process_cropped_image(best_crop, model, transform, device)

# Print or use the result from the model
print(cropped_tensor)
